In [1]:
include("./src/REoptLite.jl")
using JuMP
using Xpress
using Main.REoptLite
using JSON


┌ Info: Xpress: Found license file C:\xpressmp\bin\xpauth.xpr
└ @ Xpress C:\Users\jchang2\.julia\packages\Xpress\kcAft\src\license.jl:42
┌ Info: Xpress: Development license detected.
└ @ Xpress C:\Users\jchang2\.julia\packages\Xpress\kcAft\src\license.jl:84


In [2]:
function run(scenario, fname)
    
    m = Model(optimizer_with_attributes(Xpress.Optimizer, "logfile" => "output.log"))
    results = run_reopt(m, REoptInputs(scenario))
    
    open(fname*".json", "w") do f
        JSON.print(f, results)
    end
    
    return results
end

run (generic function with 1 method)

In [3]:
scenario_template = JSON.parsefile("test/scenarios/STM_campus_existing_sseb_crit.json")

Dict{String,Any} with 7 entries:
  "ElectricUtility" => Dict{String,Any}("outage_durations"=>Any[336],"outage_st…
  "ElectricTariff"  => Dict{String,Any}("monthly_demand_rates"=>Any[0, 0, 0, 0,…
  "Site"            => Dict{String,Any}("latitude"=>39.7407,"min_resil_timestep…
  "ElectricLoad"    => Dict{String,Any}("annual_kwh"=>0.0,"doe_reference_name"=…
  "PV"              => Dict{String,Any}("array_type"=>0,"existing_kw"=>16,"max_…
  "Storage"         => Dict{String,Any}("macrs_option_years"=>5,"can_grid_charg…
  "Financial"       => Dict{String,Any}("owner_discount_pct"=>0.1,"offtaker_dis…

In [4]:
results = Dict[]

for tzero in range(1, stop=336)
    scenario_dict = copy(scenario_template)
    scenario_dict["ElectricUtility"]["outage_start_timesteps"] = collect(range(tzero, stop=8760-336, step=336))
    s = Scenario(scenario_dict)
    push!(results, run(s, "cafe_sseb_tzero"*string(tzero)))
    println("run $tzero complete")
end

┌ Info: Querying PVWatts for prodfactor with 
│   pv.name = PV
└ @ Main.REoptLite C:\Users\jchang2\Documents\REoptLite\src\core\prodfactor.jl:41
┌ Info: PVWatts success.
└ @ Main.REoptLite C:\Users\jchang2\Documents\REoptLite\src\core\prodfactor.jl:49
┌ Info: Model built. Optimizing...
└ @ Main.REoptLite C:\Users\jchang2\Documents\REoptLite\src\core\reopt.jl:264
┌ Info: REopt solved with 
│   termination_status(m) = OPTIMAL
└ @ Main.REoptLite C:\Users\jchang2\Documents\REoptLite\src\core\reopt.jl:273


run 275 complete


┌ Info: Querying PVWatts for prodfactor with 
│   pv.name = PV
└ @ Main.REoptLite C:\Users\jchang2\Documents\REoptLite\src\core\prodfactor.jl:41
┌ Info: PVWatts success.
└ @ Main.REoptLite C:\Users\jchang2\Documents\REoptLite\src\core\prodfactor.jl:49
┌ Info: Model built. Optimizing...
└ @ Main.REoptLite C:\Users\jchang2\Documents\REoptLite\src\core\reopt.jl:264
┌ Info: REopt solved with 
│   termination_status(m) = OPTIMAL
└ @ Main.REoptLite C:\Users\jchang2\Documents\REoptLite\src\core\reopt.jl:273


run 276 complete


┌ Info: Querying PVWatts for prodfactor with 
│   pv.name = PV
└ @ Main.REoptLite C:\Users\jchang2\Documents\REoptLite\src\core\prodfactor.jl:41
┌ Info: PVWatts success.
└ @ Main.REoptLite C:\Users\jchang2\Documents\REoptLite\src\core\prodfactor.jl:49
┌ Info: Model built. Optimizing...
└ @ Main.REoptLite C:\Users\jchang2\Documents\REoptLite\src\core\reopt.jl:264
┌ Info: REopt solved with 
│   termination_status(m) = OPTIMAL
└ @ Main.REoptLite C:\Users\jchang2\Documents\REoptLite\src\core\reopt.jl:273


...

run 334 complete


┌ Info: Querying PVWatts for prodfactor with 
│   pv.name = PV
└ @ Main.REoptLite C:\Users\jchang2\Documents\REoptLite\src\core\prodfactor.jl:41
┌ Info: PVWatts success.
└ @ Main.REoptLite C:\Users\jchang2\Documents\REoptLite\src\core\prodfactor.jl:49
┌ Info: Model built. Optimizing...
└ @ Main.REoptLite C:\Users\jchang2\Documents\REoptLite\src\core\reopt.jl:264
┌ Info: REopt solved with 
│   termination_status(m) = OPTIMAL
└ @ Main.REoptLite C:\Users\jchang2\Documents\REoptLite\src\core\reopt.jl:273


run 335 complete


┌ Info: Querying PVWatts for prodfactor with 
│   pv.name = PV
└ @ Main.REoptLite C:\Users\jchang2\Documents\REoptLite\src\core\prodfactor.jl:41
┌ Info: PVWatts success.
└ @ Main.REoptLite C:\Users\jchang2\Documents\REoptLite\src\core\prodfactor.jl:49
┌ Info: Model built. Optimizing...
└ @ Main.REoptLite C:\Users\jchang2\Documents\REoptLite\src\core\reopt.jl:264
┌ Info: REopt solved with 
│   termination_status(m) = OPTIMAL
└ @ Main.REoptLite C:\Users\jchang2\Documents\REoptLite\src\core\reopt.jl:273


run 336 complete


In [5]:
r = results

62-element Array{Dict,1}:
 Dict{String,Any}("net_capital_costs" => 1.13119797e6,"year_one_demand_cost" => 0.0,"lcc" => 1.561221e6,"expected_outage_cost" => 0.0,"year_one_utility_kwh" => 3.33,"year_one_export_benefit" => 0.0,"year_one_energy_cost" => 0.0,"PVtoCUR" => [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  123.921, 82.897, 25.361, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],"year_one_energy_produced_PV" => 362588.0,"PV_net_fixed_om_costs" => 30961.0…)
 Dict{String,Any}("net_capital_costs" => 1.14972969e6,"year_one_demand_cost" => 0.0,"lcc" => 1.589265e6,"expected_outage_cost" => 0.0,"year_one_utility_kwh" => 3.06,"year_one_export_benefit" => 0.0,"year_one_energy_cost" => 0.0,"PVtoCUR" => [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  130.463, 87.273, 26.941, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],"year_one_energy_produced_PV" => 380324.0,"PV_net_fixed_om_costs" => 32475.0…)
 Dict{String,Any}("net_capital_costs" => 1.171682e6,"year_one_demand_cost" => 0.0,"lcc" => 1.618809e6,"exp